In [1]:
import re
import pandas as pd

In [2]:
df = pd.read_csv('./mozilla/bug-dataset-mozilla.csv')

In [4]:
df.shape

(768335, 14)

In [3]:
df.head()

,_id,bug_id,product,description,bug_severity,dup_id,short_desc,priority,version,component,delta_ts,bug_status,creation_ts,resolution
0,ObjectId(52eaece454dc1c410c4fbc01),35,MozillaClassic,Created by (weitsang@cs.cornell.edu) on Mond...,minor,[],Navigator does not free preference hash table ...,P3,1998-03-31,XFE,2013-11-19 18:16:47 -0800,VERIFIED,1998-04-07 01:37:03 -0700,WONTFIX
1,ObjectId(52eaece454dc1c410c4fbc02),36,SeaMonkey,Created by J. Daniel Powell (dan@java-linux.or...,critical,[],Floating Point Exception on Execution,P2,Trunk,Build Config,2012-10-31 18:36:36 -0700,VERIFIED,1998-04-07 02:04:03 -0700,INVALID
2,ObjectId(52eaece454dc1c410c4fbc03),37,MozillaClassic,Created by Chen Ronghua (chenrh@usa.net) on Mo...,normal,[],Preference Dialog does not show,P2,1998-03-31,Windows FE,2000-12-25 17:53:17 -0800,VERIFIED,1998-04-07 02:20:01 -0700,FIXED
3,ObjectId(52eaece454dc1c410c4fbc04),39,MozillaClassic,Created by Chen Ronghua (chenrh@usa.net) on Mo...,normal,[],Bookmark properties leads to an Assert failed,P2,1998-03-31,Aurora/RDF BE,2013-11-19 23:42:54 -0800,VERIFIED,1998-04-07 02:34:14 -0700,WONTFIX
4,ObjectId(52eaece454dc1c410c4fbc05),42,MozillaClassic,Created by Stephan Nagy (steph8@flash.net) on...,minor,[],navigator redraw after initial startup,P2,1998-03-31,XFE,2013-07-22 06:53:51 -0700,VERIFIED,1998-04-07 05:42:04 -0700,FIXED


In [5]:
df['product'].value_counts()

Core                      205177
Firefox                   115814
SeaMonkey                  88709
Thunderbird                32551
MailNews Core              25319
                           ...  
Petri                          5
Talkilla                       3
Developer Engagement           3
mozillaignite                  3
Internet Public Policy         1
Name: product, Length: 98, dtype: int64

In [7]:
df['component'].value_counts()

General                      143312
JavaScript Engine             22084
Layout                        16550
Bookmarks & History           16203
MailNews: Message Display     14855
                              ...  
Bengali                           1
Norton 360                        1
Toolbars                          1
Tabs                              1
hg.instantbird.org                1
Name: component, Length: 1191, dtype: int64

In [8]:
df['version'].value_counts()

Trunk          331927
unspecified    272729
other           59340
1.0 Branch       5396
1.0              5363
                ...  
409                 1
2.20.7              1
2.23.1              1
1.5R4               1
BW-1.2              1
Name: version, Length: 539, dtype: int64

In [9]:
df['priority'].value_counts()

--    632333
P3     68145
P2     28773
P1     28499
P4      6493
P5      4092
Name: priority, dtype: int64

In [4]:
#retrieve the bug_ids 
bug_ids = df['bug_id'].tolist()

In [5]:
#retrieve the descriptions
bug_descriptions = df['description'].tolist()

In [6]:
#retrieve their dup_id
bug_dup_ids = df['dup_id'].tolist()

In [16]:
#doing NER
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
from tqdm import tqdm

import csv

processed_desc_1 = []
no_ner_indices = []

for i in tqdm(range(len(bug_descriptions))):
    
    desc = bug_descriptions[i]
    doc = nlp(str(desc))
    
    enc_dict = dict()
    
    for ent in doc.ents:
        enc_dict[ent.text] = ent.label_
    
    if not enc_dict:
        no_ner_indices.append(i)
    else:
        desc = re.sub('({})'.format('|'.join(map(re.escape, enc_dict.keys()))), lambda m: enc_dict[m.group()], desc)
        
    processed_desc_1.append(desc)

 18%|█████▊                           | 135041/768335 [46:40<2:58:52, 59.01it/s]

In [14]:
#processing to remove the alphanumeric characters
processed_desc_2 = []
for i in tqdm(range(len(processed_desc_1))):
    desc = processed_desc_1[i]
    processed_desc_2.append(re.sub("[^A-Za-z0-9]+", " ", str(desc)))

100%|████████████████████████████████| 768335/768335 [00:26<00:00, 28984.32it/s]


In [ ]:
#converting to lower case and tokenizing them 